Qeusta prima parte serve ad estrarre una prima vaersione grezza dei dati dal pdf

In [173]:
# [top,left,bottom,width]
box = [143.18, 11.55, 286.80, 182.88]
# fattore di conversione
fc = 28.28
         
for i in range(0, len(box)):
    box[i] /= 10
    box[i] *= fc
    
df = read_pdf("dati/Utilizzo-e-qualità-della-risorsa-idrica-in-Italia.pdf", pages='9',area=[box],output_format="json", stream=True)

#df

#questo me lo stampa in una maniera più facilmente leggibile ad un umano

La prima e la seconda riga per la mia prima tabellla almeno, corrispondono al titolo della tabella

il codice commentato sotto, serve a visualizzare le singole righe con il rispettivo contenuto in testo

In [174]:
# prima colonna
#df[0]['data'][0][0]['text'] 

# seconda colonna
#df[0]['data'][1][0]['text'] 

Crea la tabella con i miei dati

In [175]:
table = []
for i in range(0,len(df[0]['data'])):
    if df[0]['data']:
        riga = df[0]['data'][i][0]['text']
        table.append(riga)
#table 

In [176]:
import re 

tab=[]
#test=[]    --> INUTILE
fin=[]

nome_tab = re.findall("[A-Za-z]+\s[0-9]+\.[0-9]+", table[0])  #contiene il nome tabella sul pdf es. Tavola 1.1
fonte_tab = table[len(df[0]['data'])-1]    #contiene la fonte della tab (es istat)

for i in range(1,len(table)):    #cicla partendo da uno, in modo da evitare il nome della tabella
    x = re.findall("([A-Za-z]+\s+)([0-9]+|.,[0-9]+)", table[i])   #mi seleziona tutti i nomi (di regione) seguiti da numeri  
    # forse la riga sopra va raffinata xke potrebbe dare problemi in tab he hanno nomi seguiti da numeri in righe che non sono la prima ma nemmeno le regioni
    #ATT --> inoltre la RE --> ([A-Za-z]\s+)([0-9]) presente sotto in y, dovrebbe fare la stessa cosa, alivello di utilità qui,
    #quindi potrei acnhe sostituire questa con quella RE
    if x:
        tab.append(table[i])    #ora tab contien tutti i dati necessari estratti dal pdf, sono da riordinare

for j in range(0,len(tab)):
    #([A-Za-z]\s+)([0-9])    --> INUTILE
    #Y=re.findall("([A-Za-z]\s+)([0-9])", tab[j])    --> INUTILE
    y=re.search("([A-Za-z]\s+)([0-9])", tab[j])   #mi trova tutti i nomi delle regioni + gli spazi + il primo numero
    if y:    # toglie il priomo numero dal nome delle regioni ma lascia gli spazi
        val=y.span()   #trova l'estensione della parola
        nome=tab[j][0:val[1]-1]   #contiene il nome della regione (della riga)
        nr=tab[j].split(nome)     #contiene il valore della riga (+ uno spazio bianco iniziale che sotto viene omesso)
        #test.append(nr[1])    --> INUTILE
        #*********************************
        riga_fin = str(nome)+":"+str(nr[1])
        #fin.append(z)   #aggiunge il nome della regione    --> INUTILE
        #fin.append(num)   #aggiunge i valori               --> INUTILE
        fin.append(riga_fin)

#fin.append(fonte_tab)    aggiunge la fonte alla tab, vedere come gestire la cosa

print("*************************")
#test                             --> INUTILE
#ok    test.append(z)             --> INUTILE
        #test.append(y.span())    --> INUTILE
fin
#test    --> INUTILE
#tab   # già il risultato di tab lo posso dare in pasto ad openrefine e suddividere in colonne da li     --> INUTILE

*************************


['Piemonte  :160,7  415,4  49,9  38,2 - 664,2 412',
 'Valle d’Aosta/Vallée d’Aoste  :47,1  6,1 - - - 53,2 1.140',
 'Liguria  :31,0  130,4  38,0  59,0 - 258,4 449',
 'Lombardia  :276,8  1.166,1  0,8  43,1 - 1.486,9 407',
 'Trentino-Alto Adige/Südtirol  :193,0  34,0  3,5  0,6 - 231,1 599',
 'Bolzano-Bozen  :64,5  18,9 - - - 83,4 440',
 'Trento  :128,5  15,1  3,5  0,6 - 147,7 752',
 'Veneto  :179,5  472,9  64,8  2,0 - 719,2 400',
 'Friuli-Venezia Giulia  :53,2  163,6  6,3  1,3 - 224,4 502',
 'Emilia-Romagna  :32,5  290,5  107,2  57,3 - 487,6 300',
 'Toscana  :106,8  226,3  115,4  16,1 1,0 465,6 340',
 'Umbria  :44,7  70,0 - - - 114,7 352',
 'Marche  :117,3  29,5  2,3  25,0 - 174,1 308',
 'Lazio  :823,3  309,0  3,8  38,7 - 1.174,9 546',
 'Abruzzo  :229,1  44,7  8,0 - - 281,8 581',
 'Molise  :117,9  46,1 -  13,9 - 178,0 1.559',
 'Campania  :577,3  397,6 -  0,4 - 975,3 456',
 'Puglia  :0,4  72,0 -  101,4 - 173,8 117',
 'Basilicata  :57,5  4,8 -  255,1 - 317,4 1.512',
 'Calabria  :191,9  183,

In [177]:
#tab

In [178]:
import json
with open(nome_tab[0]+".json", "w") as outfile:
    json.dump(fin, outfile, indent=3)
    
# il nome della tabella corrispode a quello sul pdf (Tavola *.*)
# esporto tutto su openrefine e divido i nomi dai valori e poi uso la RE di seguito --> (["   "]+|["  "]+|[" "]+)
# il risultato è la mia tabella corretta

Non ha molto senso dividere il numero dalla regione qui visto che posso usare open refine 
inoltre per poterla fare qui, dovrei fare una RE specifica che mi riconosca i numeri fatti in una certa maniera

a gruppi di tre, divisi da un punto, e l'ultimo gruppo diviso da una virgola, molto più facile far fare ad open refine 



In [167]:
table[len(df[0]['data'])-1]

'Fonte: Istat, Censimento delle acque per uso civile'

In [ ]:
#ATT  non ha molto senso fare la divisione dei numeri qui xke posso usare openrefine
# ATT PROBABILMENTE INUTILE DOVRO' CANCELLARE TUTTA LA CELLA

import re 

tab=[]
temp=[]

for i in range(1,len(table)):
    x = re.findall("([A-Za-z]+\s+)([0-9]+|.,[0-9]+)", table[i])
    if x:
        temp.append(table[i])
        
for j in range(0,len(temp)):
    y = re.split("([0-9]+|.,[0-9]+\s*)|(\s-\s)", temp[j])
    tab.append(y)

        
print("************************")
print(tab)
print("************************")

*************************************************************************************

In [ ]:
# elimino le colonne vuote
table = []
for i in range(0,len(df)):
    riga = {}
    riga['regione'] = regioni[i]
    if df[i]['data']:
        for j in range(0,len(df[i]['data'])):
            primaColonna = df[i]['data'][j][0]['text']
            if primaColonna != '' and primaColonna != '17 operatori sanitari' and primaColonna != "Fascia d'età":
                if primaColonna == '&gt;90':
                    primaColonna = '> 90'
                secondaColonna = df[i]['data'][j][1]['text']
                riga[primaColonna] = secondaColonna
    table.append(riga)
table 

In [ ]:
# salvo i risultati su file 
import csv
nomi_colonne = table[0].keys()
with open('dati-regioni-covid.csv', 'w') as output_file:
    dict_writer = csv.DictWriter(output_file, nomi_colonne)
    dict_writer.writeheader()
    dict_writer.writerows(table)